In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import math

In [2]:
def kalmanFilter(A,B,C,u,P0,x0,y,Q,R,dT):
    x = np.empty([2,dT+1])
    x[:,0] = x0
    P = np.empty([2,2,dT+1])
    P[:,:,0] = P0
    for i in range(dT):
            
        #prediction
        xpred = A@x[:,i] + B@u[:,i]
        Ppred = A@P[:,:,i]@A.T + Q
        
        #update
        K = Ppred@C.T@inv(C@Ppred@C.T+R)
        x[:,i+1] = xpred+K@(y[:,i]-C@xpred)
        P[:,:,i+1] = (np.identity(len(A)) - K@C)@(Ppred)
    return x,P

In [3]:
def enKF(x0,Sigma0,numOfSamples,dT,A,B,u,y,Q,R,C):
    #initializing arrays
    n = len(x0)
    meanX = np.zeros([d,dT+1]) #making my mean array
    meanX[:,0] = x0
    X = (np.random.multivariate_normal(x0,Sigma0,numOfSamples)).T #making my samples
    #tranposing the samples to make a n x numOfSamples array
    for i in range(dT):
        #initializing my measured y, control inputs, and noises array
        W = np.random.normal(size = (len(A),numOfSamples))
        Z = np.random.normal(size = (len(C@x0),numOfSamples))
        
        if len(u) == 1:
            U = np.tile(u[0,i],(1,numOfSamples))
        else:
            U = np.tile(u[:,i],(numOfSamples))
        
        if len(y) == 1:
            Ymeasure = np.tile(y[0,i],(1,numOfSamples))
        else:
            Ymeasure = np.tile(y[:,i],(numOfSamples))
        
        #calculating x' and xhat'
        xprime = A@X + B@U +(Q**(1/2)@W)
        xhat = np.mean(xprime, axis=1, keepdims = True)
        
        
        Xhat = np.tile(xhat, numOfSamples)
    
        #calculating y, covariance of y and (x,y), and K
        Y = C@xprime + R**(1/2)@Z  
        
        if len(y) == 1:
            yhat = np.mean(Y, keepdims = True)
            Yhat = np.tile(yhat, (1,numOfSamples))
            
        else:
            yhat = np.mean(Y, axis = 1, keepdims = True)
            Yhat = np.tile(yhat,numOfSamples)
        
        CovY = (1/(numOfSamples-1))*(Y-Yhat)@(Y-Yhat).T
        CovXY = (1/(numOfSamples-1))*(xprime-Xhat)@(Y-Yhat).T
        K = CovXY@np.linalg.inv(CovY)
        
        #updating my x' and getting the mean
        Xi = xprime + K@(Ymeasure-Y)
        XiMean = np.mean(Xi,axis=1)
        
        #concatenating it with my meanX
        meanX[:,i+1] = XiMean[:]
        
        #new x array to use in the next loop
        X = Xi
    return meanX